In [ ]:
import time
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing

rapl_package = '/sys/class/powercap/intel-rapl:0'

def get_energy():
    uj = int(subprocess.check_output(['eumaster4hpc-powercap-get', f'{rapl_package}/energy_uj']))
    return uj / 1000000

def run_task(*args):
    subprocess.call(['eumaster4hpc-task', *args])

def benchmark_once(*args):
    time1 = time.time()
    energy1 = get_energy()
    run_task(*args)
    energy2 = get_energy()
    time2 = time.time()

    total_energy = energy2 - energy1
    total_time = time2 - time1

    return total_time, total_energy

def print_benchmark_once(*args):
    total_time, total_energy = benchmark_once(*args)
    print(f'Total time: {total_time}')
    print(f'Total energy: {total_energy} J')


def benchmark_amdahl(*args):
    x = range(1, multiprocessing.cpu_count() + 1)
    y = [benchmark_once(*args, str(i)) for i in x]
    time = [i[0] for i in y]
    energy = [i[1] for i in y]
    speedup = [0] * len(time)
    for i in range(0, len(speedup)):
        speedup[i] = time[0] / time[i]

    fig, ax = plt.subplots()
    ax.plot(x, speedup)
    ax.set(xlabel='cores', ylabel='speedup')
    ax.grid()

    fig, ax = plt.subplots()
    ax.plot(x, time)
    ax.set(xlabel='cores', ylabel='time')
    ax.grid()

    fig, ax = plt.subplots()
    ax.plot(x, energy)
    ax.set(xlabel='cores', ylabel='energy')
    ax.grid()

    plt.show()


## Sequential benchmark

In [ ]:
print_benchmark_once('sequential', '100')


## Rayon benchmark

In [ ]:
benchmark_amdahl('rayon', '1000')
